In [ ]:
# Importo le librerie necessarie per il codice

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

# Elenco dei file da leggere
files = ['attaccanti_23.xlsx', 'attaccanti_22.xlsx', 'attaccanti_21.xlsx', 'attaccanti_20.xlsx', 'attaccanti_19.xlsx']

# Faccio un dizionario per mappare i file agli anni
year_mapping = {'attaccanti_23.xlsx': 2023,'attaccanti_22.xlsx': 2022,'attaccanti_21.xlsx': 2021,'attaccanti_20.xlsx': 2020,'attaccanti_19.xlsx': 2019}


Iniziamo con un'analisi preliminare, escludendo le colonne non numeriche come quella del nome, della nazionalità e della squadra. Oltre a queste non consideriamo rapporti, percentuali ed ogni tipo di operazione fra statistiche, in quanto non sono statistiche di 'gioco' ma sono dati creati ad hoc per analisi più speicifiche su quelle particolari aree di gioco.


In [ ]:
# Questa parte di codice crea 5 figure che rappresentano il riquadro di correlazione fra le statistiche
# suddiviso per annata

correlations = []
years = []

for file in files:
    
    data_frame = pd.read_excel(file)  # Leggi il file Excel
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost'] # Creo la colonna dei duelli aerei, può essere utile per l'analisi

    # Uso .drop per rimuovere dal data frame quelle statistiche che non voglio usare in questa analisi
    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad', 'G+A', 'G-PK','SoT%',
       'G/Sh', 'G/SoT', 'G-xG','Cmp%','Succ%','Won%'], inplace=True) 

    corr_matrix = data_frame.corr()  # Calcolo la matrice di correlazione

    year = year_mapping[file]  # Ottengo l'anno dal file usando il mapping
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")  # Ottengo in output il grafico di correlazione
    fig.show()

# Inserisco questa riga di codice in modo che in ogni momento si possano visualizzare le colonne usate per l'analisi
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')


In [ ]:
# Questa parte di codice serve per conoscere, se desiderato, quali sono le statistiche meglio e peggio correlate nel frame scelto

for file in files:
    year = year_mapping[file]  # Ottengo l'anno dal file usando il mapping
    data_frame = pd.read_excel(file)  # Leggi il file Excel
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']  # Crea la colonna dei duelli

    # Rimuovi le colonne che non vuoi usare
    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad', 'G+A', 'G-PK','SoT%',
       'G/Sh', 'G/SoT', 'G-xG','Cmp%','Succ%','Won%'], inplace=True) 

    # Calcolo della matrice di correlazione
    corr_matrix = data_frame.corr()

    stat = 'Gls'
    if stat in corr_matrix.columns:
        # Ordina le correlazioni con la statistica scelta
        sorted_corr = corr_matrix[stat].sort_values(ascending=False)

        # Trova le 5 migliori e peggiori correlazioni
        best_corr = sorted_corr[1:6]  # Parto da 1 per evitare la correlazione della statistica con se stessa
        worst_corr = sorted_corr[-5:]

        print(f"Anno {year}: Le 5 statistiche meglio correlate con {stat} sono:\n{best_corr}")
        print(f"Anno {year}: Le 5 statistiche peggio correlate con {stat} sono:\n{worst_corr}")
    else:
        print(f"La statistica {stat} non è presente nel dataset.")

   

In [ ]:
# Questa parte di codice serve per conoscere, se desiderato, la correlazione di due statistiche specifiche annata per annata
# Le statistiche di cui si vuole sapere la correlazione vanno riportate al posto di Stat1 e Stat2

correlations = []

for file in files:
    # Leggi il file Excel
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

    # Rimuovi le colonne che non sono utili per l'analisi
    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad', 'G+A', 'G-PK','SoT%',
       'G/Sh', 'G/SoT', 'G-xG','Cmp%','Succ%','Won%'], inplace=True)

    # Calcola la matrice di correlazione
    corr_matrix = data_frame.corr()

    # Ottieni l'anno dal file
    year = year_mapping[file]

    # Statistiche di interesse
    stat1 = 'xA'
    stat2 = 'KP'
    
    # Verifica se entrambe le statistiche esistono nella matrice di correlazione
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append((year, correlation_value))
        print(f"Anno {year}: Correlazione tra {stat1} e {stat2} = {correlation_value}")
    else:
        correlations.append((year, None))
        print(f"Anno {year}: Una o entrambe le statistiche ({stat1}, {stat2}) non sono presenti nel dataset.")

# Creazione del DataFrame per il grafico
if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
    correlation_df = pd.DataFrame(correlations, columns=['Anno', 'Correlazione'])
    
    # Crea il grafico dell'andamento della correlazione nel tempo
    fig = px.line(correlation_df, x='Anno', y='Correlazione', title=f"Correlazione tra {stat1} e {stat2} nel tempo", markers=True)
    fig.update_xaxes(tickmode='linear', dtick=1)
    fig.show()
else:
    print("Nessun grafico disponibile: non sono state calcolate correlazioni valide.")
